## Efficiently Cleaning Text with Pandas

You can read the original article [Here](https://pbpython.com/text-cleaning.html)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

### Download csv File if not exists:

In [2]:
%%bash

url=data.iowa.gov/api/views/m3tr-qhgy/rows.csv?accessType=DOWNLOAD

wget $url -O iowa_liquor_sales.csv -nc 

echo $?

1


O arquivo “iowa_liquor_sales.csv” já existe, não será baixado.


### Install dependencies

In [3]:
%%bash

pip install pandas numpy sidetable jinja2

You should consider upgrading via the '/home/junior/repos/github.com/aquilesics/pandas/env/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
import pandas as pd 
import numpy as np
import sidetable # a package to summarize the data

In [5]:
df = pd.read_csv("iowa_liquor_sales.csv")

### How much each store purchases? and rank them from the largest to the smallest

In [ ]:
# sidetable is a shortcut to summarize the data in a readable format. 
# The alternative is doing a groupby plus additional manipulation.

df.stb.freq(['Store Name'],value="Sale (Dollars)",style=True, cum_cols=False)

### Ideally we would like to see all the sales for Hy-Vee, Costco, Sam’s, etc grouped together.

## Cleaning attempt #1

We can investigate is using .loc plus a boolean filter with the str accessor to search for the relevant string in the Store Name column.

In [ ]:
%%timeit
#case = insensitive
# regex=false to speedup

df.loc[df["Store Name"].str.contains("Hy-Vee", case=False,regex=False),"Store Group"] = 'Hy-Vee'



1.24 s ± 69.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cleaning attempt #2

Another approach that is very performant and flexible is to use np.select to run multiple matches and apply a specified value upon match

In [ ]:
store_patterns = [
    (df['Store Name'].str.contains('Hy-Vee', case=False, regex=False), 'Hy-Vee'),
    (df['Store Name'].str.contains('Central City',
                                case=False,  regex=False), 'Central City'),
    (df['Store Name'].str.contains("Smokin' Joe's",
                                case=False,  regex=False), "Smokin' Joe's"),
    (df['Store Name'].str.contains('Walmart|Wal-Mart',
                                case=False), 'Wal-Mart'),
    (df['Store Name'].str.contains('Fareway Stores',
                                case=False,  regex=False), 'Fareway Stores'),
    (df['Store Name'].str.contains("Casey's",
                                case=False,  regex=False), "Casey's General Store"),
    (df['Store Name'].str.contains("Sam's Club", case=False,  regex=False), "Sam's Club"),
    (df['Store Name'].str.contains('Kum & Go',  regex=False, case=False), 'Kum & Go'),
    (df['Store Name'].str.contains('CVS',  regex=False, case=False), 'CVS Pharmacy'),
    (df['Store Name'].str.contains('Walgreens',  regex=False, case=False), 'Walgreens'),
    (df['Store Name'].str.contains('Yesway',  regex=False, case=False), 'Yesway Store'),
    (df['Store Name'].str.contains('Target Store',  regex=False, case=False), 'Target'),
    (df['Store Name'].str.contains('Quik Trip',  regex=False, case=False), 'Quik Trip'),
    (df['Store Name'].str.contains('Circle K',  regex=False, case=False), 'Circle K'),
    (df['Store Name'].str.contains('Hometown Foods',  regex=False,
                                case=False), 'Hometown Foods'),
    (df['Store Name'].str.contains("Bucky's", case=False,  regex=False), "Bucky's Express"),
    (df['Store Name'].str.contains('Kwik', case=False,  regex=False), 'Kwik Shop')
]

store_criteria, store_values = zip(*store_patterns)
df['Store_Group_1'] = np.select(store_criteria, store_values, 'other')

In [ ]:
df.stb.freq(['Store_Group_1'], value='Sale (Dollars)', style=True, cum_cols=False)

,Store_Group_1,Sale (Dollars),percent
0,Hy-Vee,"38,101,951",34.60%
1,Sam's Club,"8,251,870",7.49%
2,Fareway Stores,"7,028,468",6.38%
3,Wal-Mart,"6,222,725",5.65%
4,Casey's General Store,"4,044,753",3.67%
5,Central City,"3,946,233",3.58%
6,Kum & Go,"1,834,402",1.67%
7,Costco Wholesale #788 / WDM,"1,335,014",1.21%
8,Target,"1,043,642",0.95%
9,Lot-A-Spirits,"1,008,874",0.92%


In [ ]:
# This uses the combine_first function to fill in all the None values with the Store Name . 
# This is a handy trick to keep in mind when cleaning your data.

df['Store_Group_1'] = np.select(store_criteria, store_values, None)
df['Store_Group_1'] = df['Store_Group_1'].combine_first(df['Store Name'])